In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
#Read all data.

main_puf = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_TRAUMA.csv", index_col = 'inc_key', low_memory = False)
preexisting_conditions = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_PREEXISTINGCONDITIONS_PIVOT.csv", index_col = 'Inc_Key', low_memory = False)
hospital_events = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_HOSPITALEVENTS_PIVOT.csv", index_col = 'Inc_Key', low_memory = False)
ais = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_AISDIAGNOSIS_PIVOT.csv", index_col = 'inc_key', low_memory = False)
ecode = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_ECODE.csv", index_col = 'inc_key', low_memory = False)
interventions = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_INTERVENTIONS.csv", index_col = 'Inc_Key', low_memory = False)

In [ ]:
data = pd.concat([main_puf, preexisting_conditions, hospital_events, ais, ecode, interventions], axis=1)
data.columns = map(str.upper, data.columns)
data.shape

#Inclusion and Exclusion Criteria

In [ ]:
#Identify the patient population.

data = data[(data['TOTALGCS'] <= 12)]
data = data[(data['VERIFICATIONLEVEL'] == 1) | (data['VERIFICATIONLEVEL'] == 2)]

included = data.shape[0]

print('Number of patients included: ', included)

In [ ]:
#Exclude pediatric patients.

before = data.shape[0]
data = data[(data['AGEYEARS'] >= 14)]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Exclude AIS injury severity score ≥ 3 to neck.

before = data.shape[0]
data = data[data['AISSEVERITY3'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Exclude AIS injury severity score ≥ 3 to thorax.

before = data.shape[0]
data = data[data['AISSEVERITY4'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Exclude AIS injury severity score ≥ 3 to abdomen.

before = data.shape[0]
data = data[data['AISSEVERITY5'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Exclude AIS injury severity score ≥ 3 to spine.

before = data.shape[0]
data = data[data['AISSEVERITY6'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Exclude AIS injury severity score ≥ 3 to upper extremity.

before = data.shape[0]
data = data[data['AISSEVERITY7'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Exclude AIS injury severity score ≥ 3 to lower extremity.

before = data.shape[0]
data = data[data['AISSEVERITY8'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Exclude AIS injury severity score ≥ 3 to unspecified body regions.

before = data.shape[0]
data = data[data['AISSEVERITY9'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Exclude patients dead on arrival (SBP=0).

before = data.shape[0]
data = data[data['SBP'] != 0]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Exclude patients with advanced directive limiting care.

before = data.shape[0]
data = data[data['CC_ADLC'] != 'Yes']
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Drop patients with invalid GCS.

before = data.shape[0]
data = data[data['PMGCSQ_VALID'] == 1]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Drop patients with unknown GCS.

before = data.shape[0]
data = data[data['TOTALGCS'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Drop patients with unknown GCS-Motor.

before = data.shape[0]
data = data[data['GCSMOTOR'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Drop patients with unknown GCS-Verbal.

before = data.shape[0]
data = data[data['GCSVERBAL'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Drop patients with unknown GCS-Eye.

before = data.shape[0]
data = data[data['GCSEYE'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Drop patients with unknown pupillary response.

before = data.shape[0]
data = data[data['TBIPUPILLARYRESPONSE'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

In [ ]:
#Save data.

data.to_csv('/content/drive/MyDrive/TQP-MOST/2020_impact.csv')